In [60]:
import os
import pickle
import numpy as np
from datetime import datetime
from PIL import Image#, ImageDraw, ImageFilter, ImageEnhance
from time import localtime, strftime, time
from tensorflow.keras import datasets, layers, models

In [43]:
now = datetime.now()
def current_time():
    '''Help: Returns the current time as a nice string.'''
    #return strftime("%B %d, %-I:%M%p", localtime())
    return now.strftime("%H:%M:%S")

def elapsed_time(start_time):
    '''Using seconds since epoch, determine how much time has passed since the provided float. Returns string
    with hours:minutes:seconds'''
    elapsed_seconds = time()-start_time
    h = int(elapsed_seconds/3600)
    m = int((elapsed_seconds-h*3600)/60)
    s = int((elapsed_seconds-m*60)-h*3600)
    return f'{h}hr {m}m {s}s'

In [44]:
#simple function to pickle variables for later use. save a local pickle
def save_object(obj, filename, verbose=True):
    '''Help: Given an object & filepath, store the object as a pickle for later use.'''
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    if verbose:
        print(f"File saved at {filename}")
    
#and later load the file back into a variable
def load_object(filename, verbose=True):
    '''Help: Loads something previously pickled from the provided file path.'''
    with open(filename, 'rb') as f:
        load_test = pickle.load(f)
    if verbose:
        print(f"File loaded from {filename}")
    return load_test

In [45]:
data_model=load_object('Image_Sample Arrays.p')

File loaded from Image_Sample Arrays.p


In [46]:
data_of_trained_cards=load_object('data_Of_Trained_Cards.p')

File loaded from data_Of_Trained_Cards.p


In [47]:
len(data_of_trained_cards)

10

In [48]:
def train_CNN_model(model_name, model_data, unique_printings, epochs=10, verbose=True):
    '''Help: Create and train a CNN model for the provided model_data'''
    
    #unpack the model_data variable
    ((training_images, training_labels), (testing_images, testing_labels)) = model_data

    if verbose:
        print(f'Initializing {model_name} on {current_time()} ...')
        model_start_time = time()

    #if the folder already exists, delete it so we can start fresh
    if os.path.exists(f'{model_name}.model'):
        shutil.rmtree(f'{model_name}.model')

    #initialize the neural network model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=training_images.shape[1:]))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(unique_printings, activation='softmax'))

    #compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    if verbose:
        print('Network compiled, fitting data now ... \n')
    #fit the model to the provided data
    model.fit(training_images, training_labels, epochs=epochs, validation_data=(testing_images, testing_labels))

    if verbose:
        print('\nModel fit, elvaluating accuracy and saving locally now ... \n')
    #evaluate the model

    loss, accuracy = model.evaluate(testing_images, testing_labels)
    print(f'Loss: {loss}')
    print(f'Accuracy: {accuracy}')

    #save it locally for future reuse
    model.save(f'{model_name}.model')

    if verbose:
        print(f"\nModel evaluated & saved locally at '{model_name}.model' on {current_time()} \
after {elapsed_time(model_start_time)}!\n")

    return model

In [49]:
model_name='ID_ML_Model'
size=len(data_of_trained_cards)
epochs=10
verbose=True
demo_model =train_CNN_model(model_name, data_model, size,epochs, verbose)

Initializing ID_ML_Model on 19:30:14 ...
Network compiled, fitting data now ... 

Epoch 1/10
4/4 [==============================] - 3s 696ms/step - loss: 5.9858 - accuracy: 0.1500 - val_loss: 2.1810 - val_accuracy: 0.1500
Epoch 2/10
4/4 [==============================] - 3s 626ms/step - loss: 2.1212 - accuracy: 0.2400 - val_loss: 1.7246 - val_accuracy: 0.7500
Epoch 3/10
4/4 [==============================] - 3s 640ms/step - loss: 1.4969 - accuracy: 0.6900 - val_loss: 1.3348 - val_accuracy: 0.4500
Epoch 4/10
4/4 [==============================] - 3s 611ms/step - loss: 1.0023 - accuracy: 0.6700 - val_loss: 0.5672 - val_accuracy: 0.9000
Epoch 5/10
4/4 [==============================] - 3s 619ms/step - loss: 0.4198 - accuracy: 0.9100 - val_loss: 0.4473 - val_accuracy: 0.8500
Epoch 6/10
4/4 [==============================] - 3s 605ms/step - loss: 0.2007 - accuracy: 0.9700 - val_loss: 0.1854 - val_accuracy: 0.9500
Epoch 7/10
4/4 [==============================] - 3s 608ms/step - loss: 0.1046

INFO:tensorflow:Assets written to: ID_ML_Model.model\assets


INFO:tensorflow:Assets written to: ID_ML_Model.model\assets



Model evaluated & saved locally at 'ID_ML_Model.model' on 19:30:14 after 0hr 0m 30s!

